In [7]:
# Comentario especial para ver si sincroniza  
 
import pyodbc   
 
def obtenerObjetos(propietario, tipoObjeto):

    lista_tablas = []
    propietario_id = ""
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()
    
    sql_usuarios  = "select * from sysusers where name=" + "'" +propietario+ "'"
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        propietario_id=row.suid

    sql_tablas = "select * from systable where table_type='" + tipoObjeto + "' and creator=" + str(propietario_id) + " order by table_name"
    cursor.execute(sql_tablas)
    rows = cursor.fetchall()
    for row in rows:
        tbl={}
        tbl["table_creator"] = propietario
        tbl["table_id"] = row.table_id
        tbl["table_count"]= row.count
        tbl["table_name"]= row.table_name
        lista_tablas.append(tbl)

    cursor.close()
    conexionDB.close()

    return lista_tablas

In [2]:
lista_tablas = obtenerObjetos('PRG', 'BASE')
lista_vistas = obtenerObjetos('PRG', 'VIEW')

file = open("Tablas.txt", "w")
file.write('\n'.join(str(lt) for lt in lista_tablas))
file.close()

file = open("Vistas.txt", "w")
file.write(('\n'.join(str(lt) for lt in lista_vistas)))
file.close()

In [3]:
def obtenerColumnas(propietario, tabla):
    columnas = []
 
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()

    sql_usuarios  = "select * from syscolumns where tname=" + "'" + tabla + "'" + " order by colno"
    
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()
    for row in rows:
        obj = {}
        obj["col_name"]=row.cname
        obj["col_type"]=row.coltype
        obj["col_null"]=row.nulls
        obj["col_length"]=row.length
        obj["col_inPK"]=row.in_primary_key
        columnas.append(obj)

    cursor.close()
    conexionDB.close()

    return columnas

In [4]:
def generaCREATE(propietario,tabla,columnas):
    sentenciaDLL = ""
    
    sentenciaDLL = "CREATE TABLE " + "\"" + propietario + "\".\"" + tabla +"\"\n" 
    sentenciaDLL = sentenciaDLL + "(" +"\n" 
    for col in columnas:
        sentenciaDLL = sentenciaDLL + "\t" + col["col_name"] 
        if col["col_length"] != "": 
            if col["col_type"] != "long varchar":
                sentenciaDLL = sentenciaDLL + " " + col["col_type"] + "(" + str(col["col_length"]) + ")" 
            else:
                sentenciaDLL = sentenciaDLL + " " + col["col_type"]
        if col["col_null"] == "N":
            sentenciaDLL = sentenciaDLL + " NOT NULL" +",\n"
        else:
            sentenciaDLL = sentenciaDLL + ",\n"

    sentenciaDLL =  sentenciaDLL+ ")" + ";\n\n"

    return sentenciaDLL

In [5]:
todas_columnas =[]

file = open("PyUnload.sql", "a")
for col in lista_tablas:
    columnas = obtenerColumnas(col["table_creator"], col["table_name"])
    sentenciaCreate = generaCREATE(col["table_creator"], col["table_name"], columnas)
    file.write(sentenciaCreate)

    todas_columnas.append("\n" + col["table_name"])
    todas_columnas.append('\n'.join(str(x) for x in columnas))
file.close()

file = open("Columnas.txt", "w")
file.write(('\n'.join(str(x) for x in todas_columnas)))
file.close()


In [6]:
def exportData(table_name, table_id):

    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()
    
    sql  = "select * from " + table_name
    cursor.execute(sql)
    rows = cursor.fetchall()
    archivo_dat =  str(table_id) + ".dat"
    
    if len(rows)>0:    
        file = open(archivo_dat, "w")
        for row in rows:
            r = str(row)
            r = r.replace('(','')
            r = r.replace(')','')
            r = r.replace('Decimal','')
            file.write(r+"\n")
        file.close()
    else:
        return archivo_dat, False
    
    return archivo_dat , True

In [7]:
#exportData("Gestores",180)

archivos_dat = []
n = 1
for col in lista_tablas:
    exportedFile, exportResult = exportData(col["table_name"], col["table_id"])
    if exportResult:
        print (exportedFile)
        archivos_dat.append(col)
        n = n + 1
    if n==8:
        break

379.dat
380.dat
385.dat
239.dat
381.dat
248.dat
858.dat


In [8]:
def insertData(dat, archivo_sql):

    print(dat)
   
    archivo_dat = str(dat["table_id"]) + '.dat'

    #print(archivo_dat)
    file = open(archivo_sql, "a")

    sentenciaInsert = "INPUT INTO " + "\"" + dat["table_creator"] + "\".\"" + dat["table_name"] +"\"\n"
    sentenciaInsert = sentenciaInsert + "FROM " +  "\"" + archivo_dat + "\"\n"  
    sentenciaInsert = sentenciaInsert +  "FORMAT ASCII\n"
    sentenciaInsert = sentenciaInsert +  "BY ORDER;\n\n"
    
    file.write(sentenciaInsert)
    file.close()

    return True

In [9]:
for dat in archivos_dat:
    insertData(dat, "PyUnload.sql")

{'table_creator': 'PRG', 'table_id': 379, 'table_count': 4, 'table_name': 'ABCObsoletos'}
{'table_creator': 'PRG', 'table_id': 380, 'table_count': 3, 'table_name': 'ABCUnidades'}
{'table_creator': 'PRG', 'table_id': 385, 'table_count': 25, 'table_name': 'AContar'}
{'table_creator': 'PRG', 'table_id': 239, 'table_count': 3, 'table_name': 'AFPs'}
{'table_creator': 'PRG', 'table_id': 381, 'table_count': 1, 'table_name': 'AlarmaCobros'}
{'table_creator': 'PRG', 'table_id': 248, 'table_count': 1, 'table_name': 'Aplicaciones'}
{'table_creator': 'PRG', 'table_id': 858, 'table_count': 12313, 'table_name': 'AuditoriaClientes'}


In [10]:
def crearIndices(table_name):
  
    indices = {} 
    conexionDB = pyodbc.connect("DSN=BaseSQL32")
    cursor = conexionDB.cursor()

    sql_usuarios  = "select * from sysindexes where tname =" + "'" + table_name + "'"   
    
    cursor.execute(sql_usuarios)
    rows = cursor.fetchall()

    for row in rows:
        if row.indextype == "Non-unique":
            colnames = rows.colnames
            columnas_indice = colnames.split(',')
            nombre_indice
    
    print (columnas_indice)

    cursor.close()
    conexionDB.close()

    return columnas_indice

In [11]:
crearIndices("Gestores")

[]


[]